# Desafio 15

In [12]:
Threads.nthreads()   # Retorna o número de threads disponíveis no Julia.
                     # Esse valor indica quantos núcleos de execução simultânea podem ser usados em loops paralelos com @threads.


2

**Exemplo 1**

In [13]:
function loop_serie(n)        # Define uma função chamada loop_serie que recebe n como número de iterações
    s = 0.0                   # Inicializa um acumulador (soma) com valor 0.0
    for i in 1:n              # Loop que vai de 1 até n (executado de forma sequencial)
        s += sqrt(i)          # Em cada iteração, calcula a raiz quadrada de i e soma ao acumulador s
    end
    return s                  # Retorna o valor final acumulado após o loop terminar
end

@time loop_serie(2_000_000)   # Mede e exibe o tempo gasto para executar loop_serie com 2 milhões de iterações


  0.004350 seconds


1.8856187900630662e9

In [14]:
using Base.Threads                # Importa o módulo nativo de threads do Julia (não é pacote externo; faz parte da linguagem)

function loop_paralelo(n)         # Define a função loop_paralelo com n iterações
    parcial = zeros(Float64, nthreads())
                                  # Cria um vetor com uma posição para cada thread.
                                  # Cada thread vai acumular sua própria soma separadamente, evitando conflitos de escrita.

    @threads for i in 1:n         # Divide automaticamente o loop entre as threads disponíveis
        parcial[threadid()] += sqrt(i)
                                  # Cada thread identifica seu ID com threadid() e acumula a raiz quadrada de i em seu próprio índice.
    end

    return sum(parcial)           # Após o loop, soma todos os acumuladores das threads e retorna o valor total final.
end

@time loop_paralelo(2_000_000)    # Mede e exibe o tempo de execução da versão paralela

  0.047411 seconds (13.11 k allocations: 705.602 KiB, 170.87% compilation time)


1.885618790063039e9

**Exemplo 2**

In [15]:
using Base.Threads                            # Importa funcionalidades nativas de multithreading do Julia

function trabalho_pesado(x)                    # Define uma função que simula um cálculo mais custoso
    s = 0.0                                    # Inicializa acumulador
    for j in 1:200                             # Loop interno executado 200 vezes (aumenta a carga computacional)
        s += sin(j) * cos(j) / (sqrt(j) + 1)   # Executa operações matemáticas relativamente pesadas
    end
    return s * sqrt(x)                         # Multiplica o resultado final pela raiz de x
    end

function serie(n)                              # Define a versão sequencial do cálculo
    s = 0.0                                    # Acumulador
    for i in 1:n                               # Loop sequencial: executado em somente 1 thread
        s += trabalho_pesado(i)                # Chama a função pesada para cada i e acumula o resultado
    end
    return s                                   # Retorna o total acumulado
end

function paralelo(n)                           # Define a versão paralela do cálculo
    parcial = zeros(Float64, Threads.nthreads())
                                               # Cria um vetor onde cada thread terá sua própria posição para acumular os resultados (evita conflitos de escrita)
    @threads for i in 1:n                      # Divide automaticamente o loop entre as threads disponíveis
        parcial[threadid()] += trabalho_pesado(i)
                                               # Cada thread acumula o resultado da função pesada em sua posição correspondente no vetor `parcial`
    end
    return sum(parcial)                        # Soma tudo para obter o resultado final equivalente ao sequencial
end

@time serie(2_000_000)                         # Mede o tempo da execução em série (1 thread)
@time paralelo(2_000_000)                      # Mede o tempo da execução em paralelo (dividido entre threads)


 12.854348 seconds
 11.573098 seconds (12.87 k allocations: 695.719 KiB, 1.18% compilation time)


1.7054858417900217e8

**Conclusão**

In [ ]:
Para comparar a execução em série e paralela, utilizei dois loops em Julia.
O loop em série foi executado em uma única thread, enquanto o loop paralelo utilizou 2 threads (conforme Threads.nthreads() retornou 2).

No primeiro exemplo, o tempo do loop paralelo não foi menor do que o tempo do loop em série. Ou seja, paralelizar nem sempre é mais rápido — depende do custo de cada iteração.
Isso ocorreu porque cada iteração da função exigiu pouco processamento por ser um cálculo simples, e o número de threads disponível é pequeno.
Assim, o custo de coordenar as threads superou o ganho potencial do paralelismo obtido pela divisão do trabalho.

Já no segundo exemplo, usando um loop onde cada iteração executava uma função mais pesada, o tempo do loop paralelo foi menor do que o tempo do loop em série.

Esse resultado mostra que a paralelização compensa quando cada iteração é suficientemente pesada ou quando mais threads estão disponíveis.